## Load WRI Power Plant data from 2019 dataset (see https://datasets.wri.org/dataset/globalpowerplantdatabase) for original source

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have a local copy rooted in the S3_BUCKET : WRI/global_power_plant_database_v_1_3/global_power_plant_database.csv
### To tidy the data we factor into three tables:

* **wri_plants** (all the fixed data about each plant)
* **wri_annual_gwh** (per plant/per year annual generation in GWh)
* **wri_estimated_gwh** (per plant/per year estimated generation in GWh)

### The next step is to enrich with OS-C Factor metadata

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials and Data Commons libraries

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

Build a map and define schema mapping logic for parquet to sql

In [ ]:
import re
import pandas as pd
_wsdedup = re.compile(r"\s+")
_usdedup = re.compile(r"__+")
_rmpunc = re.compile(r"[,.()&$/+-]+")
# 63 seems to be a common max column name length
def snakify(name, maxlen=63):
    if isinstance(name, list):
        return [snakify(e) for e in name]
    w = str(name).casefold().rstrip().lstrip()
    w = w.replace("-", "_")
    w = _rmpunc.sub("", w)
    w = _wsdedup.sub("_", w)
    w = _usdedup.sub("_", w)
    w = w.replace("average", "avg")
    w = w.replace("maximum", "max")
    w = w.replace("minimum", "min")
    w = w.replace("absolute", "abs")
    w = w.replace("source", "src")
    w = w.replace("distribution", "dist")
    # these are common in the sample names but unsure of standard abbv
    #w = w.replace("inference", "inf")
    #w = w.replace("emissions", "emis")
    #w = w.replace("intensity", "int")
    #w = w.replace("reported", "rep")
    #w = w.replace("revenue", "rev")
    w = w[:maxlen] 
    return w

def snakify_columns(df, inplace=False, maxlen=63):
    icols = df.columns.to_list()
    ocols = snakify(icols, maxlen=maxlen)
    scols = set(ocols)
    if (len(set(ocols)) < len(ocols)):
        raise ValueError("remapped column names were not unique!")
    rename_map = dict(list(zip(icols,snakify(icols))))
    return df.rename(columns=rename_map, inplace=inplace)

rename_year_columns={}
for y in range(1900,2100):
    rename_year_columns[str(y)] = 'y{yr}'.format(yr=y)
#rename_year_columns

_p2smap = {
    'object': 'varchar',
    'string': 'varchar',
    'str': 'varchar',
    'float32': 'real',
    'Float32': 'real',
    'float64': 'double',
    'Float64': 'double',
    'int32': 'integer',
    'Int32': 'integer',
    'int64': 'bigint',
    'Int64': 'bigint',
    'category': 'varchar',
    'datetime64[ns, UTC]': 'timestamp',
    'datetime64[ns]': 'timestamp'
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create an S3 client.  We will user later when we write out data and metadata
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

schemaname = 'wri_gppd_md'  # Add the _md so we don't disturb others who are trying to use a "stable" version of wri_gppd right now
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(df)
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, '/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid))
    # df.to_parquet('/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, index=False))
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{uuid}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid),
        Filename='/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid)
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    print('dropping table: ' + tablename)
    cur.fetchall()
    
    schema = generate_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/{uuid}'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename,uuid=ingest_uuid)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

Load WRI data file using pandas *read_csv* and using *ingest_uuid* as the global UUID for this ingestion

# Do we really want to add UUID to every row, or better to put into table-level metadata?

In [ ]:
import pandas as pd
import numpy as np

wri_file = bucket.Object('WRI/global_power_plant_database_v_1_3/global_power_plant_database.csv').get()['Body']

# Because NaN cannot be converted to int type, we cannot use int as a datatype for years that are NaN

wri_df = pd.read_csv(wri_file, dtype={'latitude':'float64', 'longitude':'float64', 'capacity_mw':'float64', 'other_fuel3':'str'})

display(wri_df.columns)

### Melt the generation data into a more tidy format, dropping NA values

In [ ]:
wri_plants = wri_df[['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data', 'generation_data_source',
#        'uuid'
    ]]

# wri_id_vars = ['gppd_idnr', 'uuid']
wri_id_vars = ['gppd_idnr']

wri_value_vars = ['generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016',
                  'generation_gwh_2017', 'generation_gwh_2018', 'generation_gwh_2019']
wri_annual_gwh = wri_df.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='generation_gwh')
wri_annual_gwh['year'] = pd.to_numeric(wri_annual_gwh['year'].apply(lambda x: int(x.split('_')[-1])))
wri_annual_gwh.dropna(subset=['generation_gwh'],inplace=True)

# Push uuid to the end of the column list
# wri_annual_gwh.insert(len(wri_annual_gwh.columns)-1, 'uuid', wri_annual_gwh.pop('uuid'))

# display(wri_annual_gwh)

### Create and store metadata for annual_gwh table (1/3)

In [ ]:
custom_meta_fields = {
    'year': { 'description': 'year of report', 'dimension': 'year'},
    'gppd_idnr': { 'description': 'unique index into plants table', 'dimension': None},
    'generation_gwh': { 'description': 'electricity generation in gigawatt-hours reported for the year', 'dimension': 'GWh'}
}

custom_meta_content = {
    'title': 'WRI GPPD Annual Generation Data (GWh)',
    'description': 'A tidy representation of annual generation data from the WRI GPPD database',
    'version': '1.3.0',
    'release_date': '20210602'
    # How should we describe our transformative step here?
}

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
tablename = 'annual_gwh'
create_trino_pipeline (s3, schemaname, tablename, wri_annual_gwh, custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

Merge the estimation data so that estimates and notes are 1:1 together, dropping NA values

In [ ]:
# wri_id_vars = ['gppd_idnr', 'uuid']
wri_id_vars = ['gppd_idnr']
wri_value_vars = ['estimated_generation_gwh_2013', 'estimated_generation_gwh_2014', 'estimated_generation_gwh_2015',
                  'estimated_generation_gwh_2016', 'estimated_generation_gwh_2017']
wri_estimated_gwh = wri_df.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='estimated_generation_gwh')
wri_estimated_gwh['year'] = pd.to_numeric(wri_estimated_gwh['year'].apply(lambda x: int(x.split('_')[-1])))

wri_value_vars = ['estimated_generation_note_2013', 'estimated_generation_note_2014', 'estimated_generation_note_2015',
                  'estimated_generation_note_2016', 'estimated_generation_note_2017']
wri_estimated_note = wri_df.melt(wri_id_vars, wri_value_vars, var_name='year', value_name='estimated_generation_note')
wri_estimated_note['year'] = pd.to_numeric(wri_estimated_note['year'].apply(lambda x: int(x.split('_')[-1])))

# Push uuid to the end of the column list
# wri_estimated_gwh.insert(len(wri_estimated_gwh.columns)-1, 'uuid', wri_estimated_gwh.pop('uuid'))

display(wri_estimated_gwh)

# wri_estimated_gwh = wri_estimated_gwh.merge(wri_estimated_note, on=['gppd_idnr', 'year', 'uuid'], validate="one_to_one")
wri_estimated_gwh = wri_estimated_gwh.merge(wri_estimated_note, on=['gppd_idnr', 'year'], validate="one_to_one")

wri_estimated_gwh.dropna(subset=['estimated_generation_gwh'],inplace=True)

# Push uuid to the end of the column list
# wri_estimated_gwh.insert(len(wri_estimated_gwh.columns)-1, 'uuid', wri_estimated_gwh.pop('uuid'))

display(wri_estimated_gwh)

### Create and store metadata for estimated_gwh table

Convert the DataFrame to an Arrow table using PyArrow and inspect the table’s metadata property 

Create custom meta data and key

In [ ]:
custom_meta_fields = {
    'year': { 'description': 'year of report', 'dimension': 'year'},
    'gppd_idnr': { 'description': 'unique index into plants table', 'dimension': None},
    'estimated_generation_gwh': { 'description': 'estimated electricity generation in gigawatt-hours reported for the year', 'dimension': 'GWh'},
    'estimated_generation_note': { 'description': 'type of generation estimated', 'dimension': None}
}

custom_meta_content = {
    'title': 'WRI GPPD Estimated Generation Data (GWh)',
    'description': 'A tidy representation of estimated generation data from the WRI GPPD database',
    'version': '1.3.0',
    'release_date': '20210602'
    # How should we describe our transformative step here?
}

In [ ]:
tablename = 'estimated_gwh'
create_trino_pipeline (s3, schemaname, tablename, wri_estimated_gwh, custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{tname}.parquet'.format(sname=schemaname,tname=tablename,uuid=ingest_uuid)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

### Finally, write out plants with metadata

Create the actual metadata for the source.  In this case, it is WRI_GPPD.
The quoted text comes from the README.txt file that comes with the dataset.

In [ ]:
custom_meta_content = {}
metadata_text = """Title: Global Power Plant Database
Description: A comprehensive, global, open source database of power plants
Version: 1.3.0
Release Date: 2021-06-02
URI: http://datasets.wri.org/dataset/globalpowerplantdatabase
Copyright: Copyright 2018-2021 World Resources Institute and Data Contributors
License: Creative Commons Attribution 4.0 International -- CC BY 4.0
Contact: powerexplorer@wri.org
Citation: Global Energy Observatory, Google, KTH Royal Institute of Technology in Stockholm, Enipedia, World Resources Institute. 2019. Global Power Plant Database. Published on Resource Watch and Google Earth Engine. http://resourcewatch.org/ https://earthengine.google.com/  """

for line in metadata_text.split('\n'):
    k, v = line.split(':', 1)
    k = snakify(k)
    custom_meta_content[k] = v

custom_meta_content['abstract'] = """An affordable, reliable, and environmentally sustainable power sector is central to modern society.
Governments, utilities, and companies make decisions that both affect and depend on the power sector.
For example, if governments apply a carbon price to electricity generation, it changes how plants run and which plants are built over time.
On the other hand, each new plant affects the electricity generation mix, the reliability of the system, and system emissions.
Plants also have significant impact on climate change, through carbon dioxide (CO2) emissions; on water stress, through water withdrawal and consumption; and on air quality, through sulfur oxides (SOx), nitrogen oxides (NOx), and particulate matter (PM) emissions.

The Global Power Plant Database is an open-source open-access dataset of grid-scale (1 MW and greater) electricity generating facilities operating across the world.

The Database currently contains nearly 35000 power plants in 167 countries, representing about 72% of the world's capacity.
Entries are at the facility level only, generally defined as a single transmission grid connection point.
Generation unit-level information is not currently available. 
The methodology for the dataset creation is given in the World Resources Institute publication "A Global Database of Power Plants" [0].
Associated code for the creation of the dataset can be found on GitHub [1].
See also the technical note published in early 2020 on an improved methodology to estimate annual generation [2].

To stay updated with news about the project and future database releases, please sign up for our newsletter for the release announcement [3].


[0] www.wri.org/publication/global-power-plant-database
[1] https://github.com/wri/global-power-plant-database
[2] https://www.wri.org/publication/estimating-power-plant-generation-global-power-plant-database
[3] https://goo.gl/ivTvkd"""
custom_meta_content['name'] = 'WRI_GPPD'

Create the metadata for all the fields in all the tables

In [ ]:
field_text = """`country` (text): 3 character country code corresponding to the ISO 3166-1 alpha-3 specification [https://www.iso.org/iso-3166-country-codes.html]
`country_long` (text): longer form of the country designation
`name` (text): name or title of the power plant, generally in Romanized form
`gppd_idnr` (text): 10 or 12 character identifier for the power plant
`capacity_mw` (number): electrical generating capacity in megawatts
`latitude` (number): geolocation in decimal degrees; WGS84 (EPSG:4326)
`longitude` (number): geolocation in decimal degrees; WGS84 (EPSG:4326)
`primary_fuel` (text): energy source used in primary electricity generation or export
`other_fuel1` (text): energy source used in electricity generation or export
`other_fuel2` (text): energy source used in electricity generation or export
`other_fuel3` (text): energy source used in electricity generation or export
`commissioning_year` (number): year of plant operation, weighted by unit-capacity when data is available
`owner` (text): majority shareholder of the power plant, generally in Romanized form
`source` (text): entity reporting the data; could be an organization, report, or document, generally in Romanized form
`url` (text): web document corresponding to the `source` field
`geolocation_source` (text): attribution for geolocation information
`wepp_id` (text): a reference to a unique plant identifier in the widely-used PLATTS-WEPP database.
`year_of_capacity_data` (number): year the capacity information was reported
`generation_data_source` (text): attribution for the reported generation information"""

field_descs = [line.split(': ')[1] for line in field_text.split('\n')]
field_keys = [line.split(': ')[0].split(' ')[0][1:-1] for line in field_text.split('\n')]

Create custom meta data and key

In [ ]:
custom_meta_fields = {}
for k, v in zip(field_keys, field_descs):
    custom_meta_fields[k] = { 'description': v }

custom_meta_fields['capacity_mw']['dimension'] = 'MW'
custom_meta_fields['latitude']['dimension'] = 'degrees'
custom_meta_fields['longitude']['dimension'] = 'degrees'
custom_meta_fields['commissioning_year']['dimension'] = 'year'
custom_meta_fields['year_of_capacity_data']['dimension'] = 'year'
custom_meta_fields['year'] = { 'description': 'year of report', 'dimension': 'year'}
custom_meta_fields['gppd_idnr'] = { 'description': 'unique index into plants table', 'dimension': None}
custom_meta_fields['generation_gwh'] = { 'description': 'electricity generation in gigawatt-hours reported for the year', 'dimension': 'GWh'}
custom_meta_fields['estimated_generation_gwh'] = { 'description': 'estimated electricity generation in gigawatt-hours reported for the year', 'dimension': 'GWh'}
custom_meta_fields['estimated_generation_note'] = { 'description': 'label of the model/method used to estimate generation for the year', 'dimension': None }
custom_meta_key_fields = 'metafields'

custom_meta_content = {
    'title': 'Global Power Plant Database',
    'description': 'A comprehensive, global, open source database of power plants',
    'version': '1.3.0',
    'release_date': '20210602'
}
custom_meta_key = 'metaset'

In [ ]:
tablename = 'plants'
create_trino_pipeline (s3, schemaname, tablename, wri_plants, custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{tname}.parquet'.format(sname=schemaname,tname=tablename,uuid=ingest_uuid)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

In [ ]:
display(wri_plants)

In [ ]:
# Everything below here is speculative / in process of design

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'wri_gppd_md',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':uid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta